In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from time import time

batch_size = 64
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

Using TensorFlow backend.


In [28]:
from __future__ import print_function
import keras
from keras.models import  Model
from keras.layers import Dense, Dropout, Input, Flatten, PReLU, LeakyReLU
from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from time import time
import time as T
import traceback
import logging
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from GA.geneticAlgorithm import GenerationalGA
from GA.parentSelector.parentSelector import RandomParentSelector, LinealOrder, TournamentSelection
from GA.parentSelector.parentSelector import WheelSelection, LinealOrderII
from utils.datamanager import DataManager
#import tensorflow as tf

class Layer(object):
    def cross(self, other_layer):
        raise NotImplementedError

    def mutate(self):
        raise NotImplementedError

    def compare(self, other_layer):
        raise self.__repr__() == other_layer.__repr__()

    def self_copy(self):
        raise NotImplementedError

    def random_layer(self):
        raise NotImplementedError

    def __repr__(self):
        raise NotImplementedError
    
class NNLayer(Layer):
    def __init__(self, units=128, activation='relu', dropout=0):
        self.type = 'NN'
        self.posible_activations = ['relu', 'sigmoid', 'tanh', 'elu', 'prelu', 'leakyreLu']
        assert activation in self.posible_activations
        
        # parameters
        self.activation = activation
        self.dropout = dropout
        self.units = units
        
        self.units_lim = 1024
        self.units_prob = 0.2
        self.act_prob = 0.2
        self.drop_prob = 0.2

    def cross(self, other_layer):
        assert self.type == other_layer.type
        new_units = self.cross_units(other_layer.units)
        new_activation = self.cross_activation(other_layer.activation)
        new_dropout = self.cross_dropout(other_layer.dropout)
        return NNLayer(new_units, new_activation, new_dropout)

    def cross_activation(self, other_activation):
        if np.random.rand() > 0.5:
            return self.activation
        return other_activation

    def cross_dropout(self, other_dropout):
        b = np.random.rand()
        return self.dropout * (1 - b) + b * other_dropout

    def cross_units(self, other_units):
        b = np.random.rand()
        return int(self.units * (1 - b) + other_units * b)

    def mutate(self):
        aleatory = np.random.rand(4)
        if aleatory[0] < self.units_prob:
            self.units = np.random.randint(1, self.units_lim)
        if aleatory[1] < self.act_prob:
            self.activation = random.choice(self.posible_activations)
        if aleatory[2] < self.drop_prob:
            self.dropout = np.random.rand()

    '''
    def compare(self, other_layer):
        if self.units != other_layer.units:
            return False
        if self.activation != other_layer.activation:
            return False
        if self.dropout != other_layer.dropout:
            return False
        return True
    '''

    def self_copy(self):
        return NNLayer(self.units, self.activation, self.dropout)

    def random_layer(self):
        units = np.random.randint(1, self.units_lim)
        act = random.choice(self.posible_activations)
        drop = np.random.rand()
        return NNLayer(units, act, drop)

    def __repr__(self):
        return "%s|U:%d|A:%s|D:%0.3f" % (self.type, self.units, self.activation, self.dropout)
    
class CNNLayer(object):
    def __init__(self, filters=32, kernel_size=(3,3), activation='relu', dropout=0, maxpool=True):
        self.type = 'CNN'
        self.posible_activations = ['relu', 'sigmoid', 'tanh', 'elu', 'prelu', 'leakyreLu']
        assert activation in self.posible_activations
        
        # Parameters
        self.activation = activation
        self.dropout = dropout
        self.filters = filters
        self.k_size = kernel_size
        self.maxpool = True
        
        self.filters_lim = 1024
        self.k_lim = 9
        self.k_prob = 0.2
        self.filter_prob = 0.2
        self.act_prob = 0.2
        self.drop_prob = 0.2
        self.maxpool_prob = 0.1

    def cross(self, other_layer):
        new_filters = self.cross_filters(other_layer.filters)
        new_ksize = self.cross_kernel(other_layer.k_size)
        new_activation = self.cross_activation(other_layer.activation)
        new_dropout = self.cross_dropout(other_layer.dropout)
        new_maxpool = self.cross_maxpool(other_layer.maxpool)
        return CNNLayer(new_filters, new_ksize, new_activation, new_dropout, new_maxpool)
    
    def cross_kernel(self, )
    
    def cross_maxpool(self, other_maxpool):
        return random.choice([self.maxpool, other_maxpool])

    def cross_activation(self, other_activation):
        return random.choice([self.activation, other_activation])
    
    def cross_dropout(self, other_dropout):
        b = np.random.rand()
        return self.dropout * (1 - b) + b * other_dropout

    def cross_filters(self, other_filters):
        b = np.random.rand()
        return int(self.filters * (1 - b) + other_filters * b)

    def mutate(self):
        aleatory = np.random.rand(6)
        if aleatory[0] < self.filter_prob:
            self.filters = np.random.randint(0, self.filters_lim)
        if aleatory[1] < self.act_prob:
            self.activation = random.choice(self.posible_activations)
        if aleatory[2] < self.drop_prob:
            self.dropout = np.random.rand()
        if aleatory[3] < self.k_prob:
            self.k_size = (self.k_size[0], np.random.randint(1, self.k_lim + 1))
        if aleatory[4] < self.k_prob:
            self.k_size = (np.random.randint(1, self.k_lim + 1), self.k_size[1])
        if aleatory[5] < self.maxpool_prob:
            self.maxpool = random.choice([True, False])
    '''
    def compare(self, other_layer):
        if self.filters != other_layer.filters:
            return False
        if self.activation != other_layer.activation:
            return False
        if self.dropout != other_layer.dropout:
            return False
        if self.k_size != other_layer.k_size:
            return False
        return True
    '''

    def self_copy(self):
        return CNNLayer(self.filters, self.k_size, self.activation, self.dropout, self.maxpool)

    def random_layer(self):
        filters = np.random.randint(1, self.filters_lim)
        k_size = tuple(np.random.randint(1, self.k_lim + 1, size=(2,)))
        act = random.choice(self.posible_activations)
        drop = np.random.rand()
        maxpool = random.choice([True, False])
        return CNNLayer(filters, k_size, act, drop, maxpool)

    def __repr__(self):
        return "%s|F:%d|K:(%d,%d)|A:%s|D:%0.3f|M:%d" % (self.type, self.filters, self.k_size[0], self.k_size[1],
                                         self.activation, self.dropout, self.maxpool)

In [31]:
a = CNNLayer()
for i in range(10):
    a.mutate()
    print(a)
print(a.cross(a))

CNN|F:32|K:(3,3)|A:relu|D:0.968|M:1
CNN|F:32|K:(3,3)|A:elu|D:0.968|M:1
CNN|F:32|K:(3,3)|A:elu|D:0.968|M:1
CNN|F:32|K:(3,3)|A:elu|D:0.968|M:1
CNN|F:32|K:(3,3)|A:relu|D:0.968|M:1
CNN|F:32|K:(5,3)|A:elu|D:0.968|M:1
CNN|F:32|K:(5,8)|A:sigmoid|D:0.968|M:1
CNN|F:342|K:(5,5)|A:sigmoid|D:0.968|M:1
CNN|F:342|K:(5,5)|A:relu|D:0.968|M:0
CNN|F:342|K:(5,5)|A:relu|D:0.792|M:1


AttributeError: 'CNNLayer' object has no attribute 'cross_kernel'

In [27]:
a.filters, a.k_size, a.activation, a.dropout, a.maxpool

(673, (9, 8), 'relu', 0, ['/bin/bash: self.maxpool: orden no encontrada'])

In [ ]:
class Cromosome(object):

    def __init__(self, layers=[], fit=None):
        assert type(layers) == list
        self.n_layers = len(layers)
        self.n_cnn_layers = self.count_cnn_layers(layers)
        self.n_nn_layers = self.n_layers - self.n_cnn_layers
        
        self.layers = layers
        self.max_nn_layers = 5
        self.max_cnn_layers = 5
        
        self.grow_prob = 0.1
        self.decrese_prob = 0.1
        self.evaluator = Fitness.get_instance()
        
    def count_cnn_layers(self, layers):
        c = 0
        for l in layers:
            if l.type == 'CNN':
                c += 1
        return c

    def set_fitness(self, fit):
        self.evaluator = fit

    def random_indiv(self):
        nn_layers = np.random.randint(0, self.max_nn_layers)
        cnn_layers = np.random.randint(0, self.max_cnn_layers)
        layers = [NNLayer().random_layer() for i in range(nn_layers)]
        layers += [CNNLayer().random_layer() for i in range(cnn_layers)]
        return Cromosome(layers)

    @staticmethod
    def simple_indiv():
        return Cromosome([Layer()])

    def cross(self, other_cromosome):
        new_layers = []

        if self.n_layers == 0:
            return other_cromosome

        n_intersection = np.random.randint(0, self.n_layers)
        for i in range(self.n_layers):
            if i < n_intersection or i >= other_cromosome.n_layers:
                new_layers.append(self.layers[i].self_copy())
            else:
                try:
                    new_layers.append(self.layers[i].cross(other_cromosome.layers[i - n_intersection]))
                except IndexError:
                    print("Problem with index %d" % i)
                    print("Intersection point at %d" % n_intersection)
                    print(len(self.layers), self.layers)
                    print(len(other_cromosome.layers), other_cromosome.layers)
                    print(len(new_layers), new_layers)
                    raise IndexError
        return Cromosome(new_layers)

    def mutate(self):
        for i in range(self.n_layers):
            self.layers[i].mutate()
        if np.random.rand() < self.grow_prob:
            if np.random.rand() < 0.5 and self.n_nn_layers < self.nn_max_layers:
            self.layers.append(Layer().random_layer())
            self.n_layers = len(self.layers)

    def equals(self, other_cromosome):
        if self.n_layers != other_cromosome.n_layers:
            return False
        for i in range(self.n_layers):
            if not self.layers[i].compare(other_cromosome.layers[i]):
                return False
        return True

    def __repr__(self):
        rep = ""
        for i in range(self.n_layers):
            rep += "%d - %s \n" % (i, self.layers[i])
        return rep

    def fitness(self):
        return self.evaluator.calc(self)
